In [1]:
import pandas as pd
import warnings
import re
import matplotlib.pyplot as plt

In [28]:
df_emosi=pd.read_csv(r'src\emotion_2.csv')

### Missval Check

In [29]:
df_emosi.isna().sum()

Tweet    0
Label    0
dtype: int64

In [30]:
# rename columns Tweet to tweet and Label to label
df_emosi.rename(columns={'Tweet':'tweet','Label':'label'},inplace=True)

### Slang dan Abreviasi

In [17]:
kamus_slang=pd.read_csv(r'src\colloquial-indonesian-lexicon.csv')
kamus_slang=kamus_slang.rename(columns = {'slang' : 'kamus_slang' , 'formal' : 'kamus_perbaikan'})

# Rekonstruksi data sebagai 'dict'
slang_mapping = dict(zip(kamus_slang['kamus_slang'], kamus_slang['kamus_perbaikan']))
kamus_singkatan = pd.read_csv(r'src\kamus_singkatan.csv', header=None, names=['sebelum_perbaikan', 'setelah_perbaikan'],delimiter=';')
singkatan_mapping=dict(zip(kamus_singkatan['sebelum_perbaikan'],kamus_singkatan['setelah_perbaikan']))

### Stopword, emoji, dan Stemmer Factory

In [20]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import  StopWordRemoverFactory
import emoji
from spacy.lang.id import Indonesian
import string

In [21]:
stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()
# List of words with negation meaning
emoji_data = emoji.EMOJI_DATA

# Remove negation words from stopwords
# stopwords = set(stopwords).difference(excluded_stopwords)
nlp = Indonesian()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [22]:
def replace_emoji_with_ascii(text, emoji_data, language='id'):
    for emoji, translations in emoji_data.items():
        if language in translations:
            text = text.replace(emoji, translations[language])
    return text

In [23]:
text_with_emoji = "Saat kamu merenungkan tentang kehilangan yang pernah kamu alami, luka-luka itu terasa kembali dalam ingatan. 💔🌼 #RememberingLoss"
a = replace_emoji_with_ascii(text_with_emoji, emoji_data, language='id')
a = a.replace(":",' ').replace('_','mask').replace('-','rus').strip()
a = re.sub(' +', ' ', a)
print(a)

Saat kamu merenungkan tentang kehilangan yang pernah kamu alami, lukarusluka itu terasa kembali dalam ingatan. patahmaskhati mekar #RememberingLoss


In [24]:
def process_tweet(tweet) :
  tweet=tweet.lower()
  # link
  tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)

  # spesifik
  # tweet = re.sub(r'\[username\]|\[url\]|\[askmf\]|\[sensitive-no\]|\[satu menit kemudian\]|\[c48\]|\[idm\]', '', tweet)

  # emoji
  # tweet=replace_emoji_with_ascii(tweet,emoji_data)
  # tweet=tweet.replace(":",' ').replace('_','mask').replace('-','rus').strip()
  # tweet=re.sub(' +', ' ', tweet)

  # tokenisasi
  tokens = tweet.split()

  tweet_tokens = []
  for ele in tokens:
    ele_kamus = kamus_singkatan.get(ele, ele)
    ele_slang = slang_mapping.get(ele_kamus, ele_kamus)
    tweet_tokens.append(ele_slang)

  tweet = ' '.join(tweet_tokens)
  tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
  # tweet = re.sub(r'#([^\s]+)', '', tweet)
  tweet=re.sub(r'\d+', '', tweet)
  tweet = tweet.strip('\'"')
  tweet = tweet.lstrip('\'"')

  tweet = "".join([char for char in tweet if char not in string.punctuation])

  doc = nlp(tweet)

  tokens = [token.text for token in doc]
      # Hapus stopwords dari tokens
  filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
  tweet = ' '.join(filtered_tokens)

  tweet=stemmer.stem(tweet)
  # tweet=tweet.replace('mask',' ').replace('rus','-')

  return tweet

In [25]:
print(process_tweet('hai sayangnya adalah 😂'))

hai sayang


In [31]:
print(process_tweet(str(df_emosi['tweet'][0])))

pagi buat emosi


In [32]:
df_emosi.head(5)

,tweet,label
0,pagi2 udah di buat emosi :),Anger
1,"kok stabilitas negara, memange 10 thn negara t...",Anger
2,dah lah emosi mulu liat emyu,Anger
3,"aib? bodoh benar! sebelum kata aib itu muncul,...",Anger
4,dih lu yg nyebelin bego,Anger


In [33]:
df_emosi['tweet'] = df_emosi['tweet'].apply(lambda x: process_tweet(str(x)))

In [37]:
df_emosi.to_csv(r'src\cleaned_emotion_2.csv', index=False)

In [36]:
df_emosi

,tweet,label
0,pagi buat emosi,Anger
1,kok stabilitas negara memange tahun negara ama...,Anger
2,deh lah emosi mulu lihat emyu,Anger
3,aib bodoh benar kata aib muncul lebih tindak k...,Anger
4,dih lu sebal bego,Anger
...,...,...
7075,pagi isi semesta sujud zat acap lupa kau sebut...,Sad
7076,meski engkau pergi meski engkau tinggal meski ...,Sad
7077,biasa kalah sakit,Sad
7078,apakabar ku baikbaik aku sedang baik fikiranku...,Sad
